In [1]:
%load_ext autoreload
%autoreload 3

In [2]:
import pandas as pd
from pandasql import sqldf 
import data_helpers as dh

In [3]:
# load tables
game_info = dh.get_table('game_info')
all_picks = dh.get_table('all_picks')
all_picks_info = dh.get_table('all_picks_info')

In [6]:
game_info.head()

,game_id,winner,week_id,away_team,home_team,away_line,home_line,favorite,underdog
0,Lions (+4) @ Chiefs [TNF],Lions,1,Lions,Chiefs,4.0,-4.0,Chiefs,Lions
1,Panthers (+3.5) @ Falcons,Falcons,1,Panthers,Falcons,3.5,-3.5,Falcons,Panthers
2,Texans (+9.5) @ Ravens,Ravens,1,Texans,Ravens,9.5,-9.5,Ravens,Texans
3,Bengals (-2) @ Browns,Browns,1,Bengals,Browns,-2.0,2.0,Bengals,Browns
4,Jaguars (-4.5) @ Colts,Jaguars,1,Jaguars,Colts,-4.5,4.5,Jaguars,Colts


In [4]:
all_picks.head()

,pick,game_id,week_id,user_id
0,Lions,Lions (+4) @ Chiefs [TNF],1,jordan
1,Falcons,Panthers (+3.5) @ Falcons,1,jordan
2,Ravens,Texans (+9.5) @ Ravens,1,jordan
3,Bengals,Bengals (-2) @ Browns,1,jordan
4,Colts,Jaguars (-4.5) @ Colts,1,jordan


In [5]:
all_picks_info.head()

,pick,game_id,week_id,user_id,winner,away_team,home_team,away_line,home_line,favorite,underdog,didnt_pick
0,Lions,Lions (+4) @ Chiefs [TNF],1,jordan,Lions,Lions,Chiefs,4.0,-4.0,Chiefs,Lions,Chiefs
1,Chiefs,Lions (+4) @ Chiefs [TNF],1,daniel,Lions,Lions,Chiefs,4.0,-4.0,Chiefs,Lions,Lions
2,Lions,Lions (+4) @ Chiefs [TNF],1,greg,Lions,Lions,Chiefs,4.0,-4.0,Chiefs,Lions,Chiefs
3,Chiefs,Lions (+4) @ Chiefs [TNF],1,adam,Lions,Lions,Chiefs,4.0,-4.0,Chiefs,Lions,Lions
4,Chiefs,Lions (+4) @ Chiefs [TNF],1,james,Lions,Lions,Chiefs,4.0,-4.0,Chiefs,Lions,Lions


## Some quick analytics / data exploration

In [7]:
# games / wins / losses
q = '''
SELECT
    user_id, 
    COUNT(CASE WHEN pick IS NOT NULL THEN 1 END) as num_games, 
    COUNT(CASE WHEN pick = winner THEN 1 END) num_wins,
    COUNT(CASE WHEN pick != winner and winner != 'PUSH' THEN 1 END) num_losses,
    COUNT(CASE WHEN winner = 'PUSH' THEN 1 END) num_pushes
FROM all_picks_info
GROUP BY 1
ORDER BY 3 DESC
'''
sqldf(q)

,user_id,num_games,num_wins,num_losses,num_pushes
0,jordan,272,140,119,13
1,james,272,138,121,13
2,greg,272,135,124,13
3,colin,272,133,126,13
4,adam,270,130,127,13
5,daniel,272,127,132,13
6,andres,272,127,132,13


In [9]:
# weekly record for a given user
q = '''
SELECT
    user_id,
    week_id,
    COUNT(*) as num_games, 
    COUNT(CASE WHEN pick = winner THEN 1 END) num_wins,
    COUNT(CASE WHEN pick != winner and winner != 'PUSH' THEN 1 END) num_losses,
    COUNT(CASE WHEN winner = 'PUSH' THEN 1 END) num_pushes
FROM all_picks_info
WHERE user_id = 'daniel'
GROUP BY 1, 2
ORDER BY week_id DESC
LIMIT 5
'''
sqldf(q)

,user_id,week_id,num_games,num_wins,num_losses,num_pushes
0,daniel,18,16,8,8,0
1,daniel,17,16,11,5,0
2,daniel,16,16,11,4,1
3,daniel,15,16,6,8,2
4,daniel,14,15,5,9,1


In [18]:
# most picked teams by person (with DESC)
# least picked teams by person (without DESC)
q = '''
SELECT
    user_id,
    pick,
    COUNT(*) as num_games
FROM all_picks_info
GROUP BY 1, 2
ORDER BY 3 DESC
'''
sqldf(q).head(10)

,user_id,pick,num_games
0,greg,Patriots,17
1,jordan,Patriots,17
2,james,Bengals,16
3,andres,Lions,15
4,daniel,Patriots,15
5,james,49ers,15
6,james,Bills,15
7,james,Chiefs,15
8,colin,Vikings,14
9,greg,Chiefs,14


In [21]:
# one user, one week
q = '''
SELECT
    *
FROM all_picks_info
where user_id = 'daniel'
and week_id = 15
'''
sqldf(q).head(5)

,pick,game_id,week_id,user_id,winner,away_team,home_team,away_line,home_line,favorite,underdog,didnt_pick
0,Chargers,Chargers (+3) @ Raiders [TNF],15,daniel,Raiders,Chargers,Raiders,3.0,-3.0,Raiders,Chargers,Raiders
1,Lions,Broncos (+5) @ Lions,15,daniel,Lions,Broncos,Lions,5.0,-5.0,Lions,Broncos,Broncos
2,Vikings,Vikings (+3) @ Bengals,15,daniel,PUSH,Vikings,Bengals,3.0,-3.0,Bengals,Vikings,Bengals
3,Colts,Steelers (+1) @ Colts,15,daniel,Colts,Steelers,Colts,1.0,-1.0,Colts,Steelers,Steelers
4,Falcons,Falcons (-3) @ Panthers,15,daniel,Panthers,Falcons,Panthers,-3.0,3.0,Falcons,Panthers,Panthers


In [22]:
# some future work -- looking at team wins over time
q = '''
SELECT
    winner,
    count(*) as num_wins
FROM game_info
where week_id < 10
GROUP BY 1
ORDER BY 2 DESC
'''
sqldf(q).head(5)

,winner,num_wins
0,PUSH,7
1,Ravens,6
2,Lions,6
3,Jaguars,6
4,Dolphins,6


## Pickem Statistics

In [31]:
# basic stats: games, wins, losses, pushes, win %
q = '''
SELECT
    user_id,
    count(case when pick is not null then 1 end) as num_games,
    count(case when pick = winner then 1 end) as num_wins,
    count(case when pick != winner and winner != 'PUSH' then 1 end) as num_losses,
    count(case when winner = 'PUSH' then 1 end) as num_pushes,
    round(count(case when pick = winner then 1 end) * 1.0 / 
        count(case when winner != 'PUSH' and pick is not null then 1 end), 3) as win_pct
FROM all_picks_info
GROUP BY 1
ORDER by win_pct DESC
'''
sqldf(q)

,user_id,num_games,num_wins,num_losses,num_pushes,win_pct
0,jordan,272,140,119,13,0.541
1,james,272,138,121,13,0.533
2,greg,272,135,124,13,0.521
3,colin,272,133,126,13,0.514
4,adam,270,130,127,13,0.506
5,daniel,272,127,132,13,0.490
6,andres,272,127,132,13,0.490


In [33]:
# money made/lost actually betting every game ($20 unit)
q = '''
SELECT
    user_id,
    round(sum(case 
        when pick = winner then 20 * 100.0/110 
        when pick != winner and winner != 'PUSH' then -20
        when pick = 'PUSH' then 0
    end), 2) as earnings
FROM all_picks_info
GROUP BY 1
order by 1
'''
sqldf(q)

,user_id,earnings
0,adam,-176.36
1,andres,-330.91
2,colin,-101.82
3,daniel,-330.91
4,greg,-25.45
5,james,89.09
6,jordan,165.45


In [39]:
# top 2 teams you bet most on to cover (rk <= 2)
# top 2 teams you bet least on to cover (change DESC to ASC)
# include how the person did when betting on the team
q = '''
WITH user_team_ranks AS (
    SELECT
        user_id,
        pick,
        count(*) num_picks,
        count(case when pick = winner then 1 end) num_win,
        count(case when pick != winner and winner != 'PUSH' then 1 end) num_loss,
        row_number() over(partition by user_id order by count(*) DESC) as rk
    FROM all_picks_info
    GROUP BY 1, 2
)
SELECT *
FROM user_team_ranks
WHERE rk <= 2
'''
sqldf(q)

,user_id,pick,num_picks,num_win,num_loss,rk
0,adam,Dolphins,12,7,5,1
1,adam,Saints,11,3,6,2
2,andres,Lions,15,10,4,1
3,andres,Patriots,12,4,8,2
4,colin,Vikings,14,6,6,1
5,colin,Saints,13,4,7,2
6,daniel,Patriots,15,5,10,1
7,daniel,Cardinals,12,6,6,2
8,greg,Patriots,17,5,12,1
9,greg,Chiefs,14,8,6,2


In [43]:
# top 2 teams you bet most against to cover (rk <= 2)
# include how the person did when betting *against* the team
# so the num_win and num_loss refers to the person's bets, not the team's record
q = '''
WITH user_team_ranks AS (
    SELECT
        user_id,
        didnt_pick,
        count(*) num_picks,
        count(case when pick = winner then 1 end) num_win,
        count(case when pick != winner and winner != 'PUSH' then 1 end) num_loss,
        row_number() over(partition by user_id order by count(*) DESC) as rk
    FROM all_picks_info
    GROUP BY 1, 2
)
SELECT *
FROM user_team_ranks
WHERE rk <= 2
'''
sqldf(q)

,user_id,didnt_pick,num_picks,num_win,num_loss,rk
0,adam,Chiefs,13,5,8,1
1,adam,Texans,12,8,3,2
2,andres,Saints,14,7,6,1
3,andres,Browns,13,5,7,2
4,colin,Bengals,15,7,7,1
5,colin,Steelers,13,5,8,2
6,daniel,Broncos,15,9,5,1
7,daniel,Falcons,12,7,4,2
8,greg,Jets,14,7,6,1
9,greg,Dolphins,14,5,9,2


In [44]:
# individual person betting
q = '''
WITH user_team_ranks AS (
    SELECT
        user_id,
        pick,
        count(*) num_picks,
        count(case when pick = winner then 1 end) num_win,
        count(case when pick != winner and winner != 'PUSH' then 1 end) num_loss,
        row_number() over(partition by user_id order by count(*) desc) as rk
    FROM all_picks_info
    WHERE user_id = 'daniel'
    GROUP BY 1, 2
)
SELECT *
FROM user_team_ranks
'''
sqldf(q)

,user_id,pick,num_picks,num_win,num_loss,rk
0,daniel,Patriots,15,5,10,1
1,daniel,Cardinals,12,6,6,2
2,daniel,Vikings,11,5,3,3
3,daniel,Seahawks,11,6,4,4
4,daniel,Chiefs,11,6,5,5
5,daniel,Buccaneers,11,7,4,6
6,daniel,Bills,11,4,6,7
7,daniel,Lions,10,6,3,8
8,daniel,Commanders,10,5,5,9
9,daniel,49ers,10,7,3,10


In [48]:
# worst team you bet with (minimum 7 games)
# best team you bet with (add DESC to the rk line)
q = '''
WITH user_team_ranks AS (
    SELECT
        user_id,
        pick,
        count(*) num_picks,
        count(case when pick = winner then 1 end) as won_pick,
        count(case when pick != winner and winner != 'PUSH' then 1 end) as won_pick,
        count(case when pick = winner then 1 end) * 1.0 / 
            count(case when winner != 'PUSH' then 1 end) as pct_won_when_picking
    FROM all_picks_info
    GROUP BY 1, 2
),
ranks AS (
    select
        *,
        row_number() over(partition by user_id order by pct_won_when_picking desc) rk
    from user_team_ranks
    where num_picks >= 7
)
select *
from ranks
where rk <= 2
'''
sqldf(q)

,user_id,pick,num_picks,won_pick,won_pick:1,pct_won_when_picking,rk
0,adam,Raiders,8,6,2,0.750000,1
1,adam,Jets,8,5,2,0.714286,2
2,andres,Cardinals,9,7,2,0.777778,1
3,andres,Raiders,9,7,2,0.777778,2
4,colin,Colts,10,7,3,0.700000,1
5,colin,Ravens,12,8,4,0.666667,2
6,daniel,Jaguars,8,6,2,0.750000,1
7,daniel,Ravens,8,6,2,0.750000,2
8,greg,Seahawks,7,5,1,0.833333,1
9,greg,Rams,11,8,3,0.727273,2


In [50]:
# TNF winners
q = '''
select 
    user_id,
    count(case when game_id like '%[TNF]%' then 1 end) num_tnf,
    count(case when game_id like '%[TNF]%' and pick = winner then 1 end) tnf_wins,
    count(case when game_id like '%[TNF]%' and pick != winner and winner != 'PUSH' then 1 end) tnf_losses
from all_picks_info
group by 1
'''
sqldf(q)

,user_id,num_tnf,tnf_wins,tnf_losses
0,adam,16,9,6
1,andres,16,8,7
2,colin,16,5,10
3,daniel,16,8,7
4,greg,16,9,6
5,james,16,5,10
6,jordan,16,10,5


In [183]:
# MNF winners
q = '''
select 
    user_id,
    count(case when game_id like '%[MNF]%' then 1 end) num_mnf,
    count(case when game_id like '%[MNF]%' and pick = winner then 1 end) mnf_wins,
    count(case when game_id like '%[MNF]%' and pick != winner and winner != 'PUSH' then 1 end) mnf_losses
from all_picks_info
group by 1
'''
sqldf(q)

,user_id,num_mnf,mnf_wins,mnf_losses
0,adam,19,9,9
1,andres,19,6,12
2,colin,19,9,9
3,daniel,19,7,11
4,greg,19,8,10
5,james,19,9,9
6,jordan,19,8,10


In [196]:
# home team pct
q = '''
SELECT
    user_id,
    count(*) num_picks,
    count(case when pick = home_team then 1 end) num_home,
    count(case when pick = away_team then 1 end) num_away,
    count(case when pick = home_team then 1 end) * 1.0 / count(*) num_home_pct
FROM all_picks_info
GROUP BY 1
'''
sqldf(q)

,user_id,num_picks,num_home,num_away,num_home_pct
0,adam,272,119,149,0.437500
1,andres,272,160,112,0.588235
2,colin,272,144,128,0.529412
3,daniel,272,71,201,0.261029
4,greg,272,128,144,0.470588
5,james,272,146,126,0.536765
6,jordan,272,151,119,0.555147


In [212]:
# fav/und pick pct
q = '''
SELECT
    user_id,
    count(*) num_picks,
    count(case when pick = favorite then 1 end) num_fav,
    count(case when pick = underdog then 1 end) num_und,
    count(case when pick = favorite then 1 end) * 1.0 / count(*) num_fav_pct
FROM all_picks_info
GROUP BY 1
'''
sqldf(q)

,user_id,num_picks,num_fav,num_und,num_fav_pct
0,adam,272,132,135,0.485294
1,andres,272,157,114,0.577206
2,colin,272,148,123,0.544118
3,daniel,272,131,140,0.481618
4,greg,272,162,109,0.595588
5,james,272,184,87,0.676471
6,jordan,272,176,93,0.647059


In [242]:
# fav/underdog pick pct by week
q = '''
SELECT
    user_id,
    week_id,
    count(*) num_picks,
    count(case when pick = favorite then 1 end) num_fav,
    count(case when pick = underdog then 1 end) num_und,
    count(case when pick = favorite then 1 end) * 1.0 / count(*) num_fav_pct,
    count(case when pick = winner then 1 end) num_wins,
    count(case when pick != winner and winner != 'PUSH' then 1 end) num_losses
FROM all_picks_info
GROUP BY 1, 2
ORDER BY 6 DESC
'''
sqldf(q).head(10)

,user_id,week_id,num_picks,num_fav,num_und,num_fav_pct,num_wins,num_losses
0,colin,12,16,14,2,0.875000,9,6
1,daniel,18,16,14,2,0.875000,8,8
2,james,1,16,14,2,0.875000,4,12
3,james,6,15,13,2,0.866667,8,7
4,james,7,13,11,2,0.846154,4,9
5,jordan,3,16,13,3,0.812500,11,4
6,jordan,4,16,13,3,0.812500,7,8
7,jordan,18,16,13,3,0.812500,9,7
8,jordan,6,15,12,3,0.800000,9,6
9,andres,9,14,11,3,0.785714,7,7


In [206]:
# home team pct
q = '''
SELECT
    count(case when home_team = winner then 1 end) num_home,
    count(case when away_team = winner then 1 end) num_away,
    count(case when home_team = winner then 1 end) * 1.0 / count(case when winner != 'PUSH' then 1 end) pct_home
FROM game_info
'''
sqldf(q)

,num_home,num_away,pct_home
0,130,129,0.501931


In [207]:
# favorite team pct
q = '''
SELECT
    count(case when favorite = winner then 1 end) num_fav,
    count(case when underdog = winner then 1 end) num_und,
    count(case when favorite = winner then 1 end) * 1.0 / count(case when winner != 'PUSH' then 1 end) pct_fav
FROM game_info
'''
sqldf(q)

,num_fav,num_und,pct_fav
0,135,123,0.521236


In [222]:
# favorite team pct
q = '''
SELECT
    user_id,
    count(case when favorite = home_team then 1 end) fav_home,
    count(case when favorite = home_team and favorite = pick then 1 end) pick_fav_home,
    count(case when favorite = home_team and favorite = pick then 1 end) * 1.0 / count(case when favorite = home_team then 1 end) pick_fav_home_pct,
    count(case when underdog = home_team then 1 end) fav_away,
    count(case when underdog = home_team and underdog = pick then 1 end) pick_und_home,
    count(case when underdog = home_team and underdog = pick then 1 end) * 1.0 / count(case when underdog = home_team then 1 end) pick_und_home_pct
FROM all_picks_info
GROUP BY 1
'''
sqldf(q)

,user_id,fav_home,pick_fav_home,pick_fav_home_pct,fav_away,pick_und_home,pick_und_home_pct
0,adam,169,75,0.443787,102,43,0.421569
1,andres,169,107,0.633136,102,52,0.509804
2,colin,169,95,0.562130,102,49,0.480392
3,daniel,169,50,0.295858,102,21,0.205882
4,greg,169,94,0.556213,102,34,0.333333
5,james,169,114,0.674556,102,32,0.313725
6,jordan,169,112,0.662722,102,38,0.372549


In [225]:
# home underdog against spread
q = '''
SELECT
    count(case when favorite = home_team then 1 end) fav_home,
    count(case when favorite = home_team and favorite = winner then 1 end) fav_home_wins,
    count(case when favorite = home_team and favorite = winner then 1 end) * 1.0 / count(case when favorite = home_team then 1 end) fav_home_wins_pct,
    count(case when underdog = home_team then 1 end) und_home,
    count(case when underdog = home_team and underdog = winner then 1 end) und_home_wins,
    count(case when underdog = home_team and underdog = winner then 1 end) * 1.0 / count(case when underdog = home_team then 1 end) und_home_wins_pct
FROM game_info
WHERE winner != 'PUSH'
'''
sqldf(q)

,fav_home,fav_home_wins,fav_home_wins_pct,und_home,und_home_wins,und_home_wins_pct
0,163,85,0.521472,95,45,0.473684


In [233]:
# best/worst weeks
q = '''
with weekly_win_pct as (
SELECT
    user_id,
    week_id,
    count(case when pick = winner then 1 end) num_wins,
    count(case when pick != winner then 1 end) num_losses,
    count(case when pick = winner then 1 end) * 1.0 / count(*) win_pct
FROM all_picks_info
WHERE winner != 'PUSH'
GROUP BY 1,2
),
weeks_rk as (
select 
    *,
    row_number() over(partition by user_id order by win_pct) rk
from weekly_win_pct
)
select *
from weeks_rk
where rk = 1
'''
sqldf(q)

,user_id,week_id,num_wins,num_losses,win_pct,rk
0,adam,8,4,10,0.285714,1
1,andres,14,2,12,0.142857,1
2,colin,7,3,10,0.230769,1
3,daniel,9,3,11,0.214286,1
4,greg,15,4,10,0.285714,1
5,james,1,4,12,0.250000,1
6,jordan,8,5,9,0.357143,1
